In [1]:
# Hate Crimes in the US
# Flask App

# Load dependencies
from flask import Flask, jsonify, render_template
from sqlalchemy import create_engine, text, func, extract, inspect, Integer
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy.sql.expression import cast

# Get database information from config file
# Note: config.py must be created locally and is not stored in Github
from config import db_username, db_password, db_host, db_port, db_name

In [2]:
# Create a SQLAlchemy database engine
db_url = f'postgresql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}'
engine = create_engine(db_url)

In [3]:
# Reflect an existing database
Base = automap_base()
Base.prepare(autoload_with=engine)

session = Session(engine)

print('Connected to database and session initiated')

Connected to database and session initiated


In [ ]:
# Inspector testing
inspector = inspect(engine)

inspector.get_table_names()

display(inspector.get_columns('bias'))
display(inspector.get_pk_constraint('bias'))
display(inspector.get_pk_constraint('incident_bias'))

In [33]:
# Define variables for related tables
I = Base.classes.incident
S = Base.classes.state
J = Base.classes.jurisdiction
E = Base.classes.race
B = Base.classes.bias
IB = Base.classes.incident_bias
O = Base.classes.offense
IO = Base.classes.incident_offense
VT = Base.classes.victim_type
IVT = Base.classes.incident_victim_type
L = Base.classes.location
IL = Base.classes.incident_location
C = Base.classes.census_data

In [52]:
# Create state list for dropdowns
state_results = session.query(S.state).all()

state_list = [{'state': row.state} for row in state_results] 

# Alternate code to include all 
# state_list = [{'state': 'All'}]
# state_list.append([{'state': row.state} for row in state_results])

state_list[:4]

[{'state': 'Alaska'},
 {'state': 'Alabama'},
 {'state': 'Arkansas'},
 {'state': 'Arizona'}]

In [53]:
# Create year list for dropdowns
year_results = session.query(
    cast(extract('year', I.incident_date), Integer).label('year')   
).group_by(
    extract('year', I.incident_date)
).order_by(
    extract('year', I.incident_date).asc()
).all()

year_list = [{'year': row.year} for row in year_results]

year_list[:4]

[{'year': 2009}, {'year': 2010}, {'year': 2011}, {'year': 2012}]

In [46]:
# Route to get data with incidents by year
# @.app.route('/annual-totals')

def get_data_annual_totals():
    try:
        
        # Query database
        results = session.query(
            cast(extract('year', I.incident_date), Integer).label('year'),
            func.count(I.incident_id).label('incidents')
        ).group_by(
            extract('year', I.incident_date)
        ).order_by(
            extract('year', I.incident_date).asc()
        ).all()

        # Convert query result to a list of dictionaries
        keys = ['year', 'incidents']
        data_list = [dict(zip(keys, row)) for row in results]

        # Create dictionary to store list
        dataToReturn = {'data': data_list}
        
        # return jsonify(dataToReturn)
        return dataToReturn   
    
    except Exception as e:
        print("Error accessing the table:", str(e))
        # return jsonify({"error": "Table access failed"}), 500

In [47]:
# Preview data from list for notebook only
data = get_data_annual_totals()

data['data'][:4]

[{'year': 2009, 'incidents': 6612},
 {'year': 2010, 'incidents': 6628},
 {'year': 2011, 'incidents': 6299},
 {'year': 2012, 'incidents': 6592}]

In [50]:
# Route to get data with incidents by year, state and bias category
# @.app.route('/bias')

def get_data_bias():
    try:
        
        # Query database
        results = session.query(
            cast(extract('year', I.incident_date), Integer).label('year'),
            S.state,
            B.bias_category,
            func.count(I.incident_id).label('incidents')
        ).join(
            S, S.state_abbr == I.state_abbr 
        ).join(
            IB, IB.incident_id == I.incident_id
        ).join(
            B, B.bias_id == IB.bias_id 
        ).group_by(
            extract('year', I.incident_date),
            S.state,
            B.bias_category
        ).order_by(
            extract('year', I.incident_date).asc()
        ).all()

        # Convert query result to a list of dictionaries
        data_list = [{'year': row.year, 'state': row.state, 'bias_category': row.bias_category,
              'incidents': row.incidents} for row in results]

        dataToReturn = {'data': data_list, 'state': state_list}
        
        # return jsonify(dataToReturn)
        return dataToReturn   
    
    except Exception as e:
        print("Error accessing the table:", str(e))
        # return jsonify({"error": "Table access failed"}), 500


In [51]:
# Preview data from list for notebook only
data = get_data_bias()

data['data'][:4]

[{'year': 2009,
  'state': 'Alabama',
  'bias_category': 'Race, Ethnicity or Ancestry',
  'incidents': 6},
 {'year': 2009,
  'state': 'Alabama',
  'bias_category': 'Religion',
  'incidents': 1},
 {'year': 2009,
  'state': 'Alabama',
  'bias_category': 'Sexual Orientation',
  'incidents': 2},
 {'year': 2009,
  'state': 'Alaska',
  'bias_category': 'Race, Ethnicity or Ancestry',
  'incidents': 7}]

In [54]:
# Route to get data with population and incident rates by year and state
# @.app.route('/pop-inc-rates')

def get_data_pop_rates():
    try:

        results = session.query(
            cast(extract('year', I.incident_date), Integer).label('year'),
            S.state,
            func.sum(C.population).label('population'),
            func.count(I.incident_id).label('incidents'), 
        ).join(
            C, C.state_abbr == S.state_abbr
        ).join(
            I, I.state_abbr == S.state_abbr
        ).filter(
            C.year == extract('year', I.incident_date) 
        ).group_by(
            extract('year', I.incident_date),
            S.state
        ).all()

        data_list = [{'year': row.year, 'state': row.state, 'incidents': row.incidents, 'population': row.population,
              'incident_rate': round(row.incidents/row.population * 10000000, 2)} for row in results]

        dataToReturn = {'data': data_list, 'state': state_list}
        
        # return jsonify(dataToReturn)
        return dataToReturn   
    
    except Exception as e:
        print("Error accessing the table:", str(e))
        # return jsonify({"error": "Table access failed"}), 500

In [55]:
# Preview data from list for notebook only
data = get_data_pop_rates()

data['data'][:4]

[{'year': 2011,
  'state': 'South Carolina',
  'incidents': 744,
  'population': 558304916,
  'incident_rate': 13.33},
 {'year': 2020,
  'state': 'Georgia',
  'incidents': 1392,
  'population': 2368713040,
  'incident_rate': 5.88},
 {'year': 2014,
  'state': 'Maryland',
  'incidents': 96,
  'population': 91020912,
  'incident_rate': 10.55},
 {'year': 2019,
  'state': 'Idaho',
  'incidents': 126,
  'population': 34819554,
  'incident_rate': 36.19}]

In [67]:
# Route to get data with population and incident rates by year and state
# @.app.route('/pop-inc-rates')

def get_data_offenses():
    try:

        results = session.query(
            cast(extract('year', I.incident_date), Integer).label('year'),
            S.state,
            O.offense,
            func.count(I.incident_id).label('incidents'),  
        ).join(
            I, I.state_abbr == S.state_abbr
        ).join(
            IO, IO.incident_id == I.incident_id
        ).join(
            O, O.offense_id == IO.offense_id
        ).group_by(
            extract('year', I.incident_date),
            S.state,
            O.offense
        ).all()

        data_list = [{'year': row.year, 'state': row.state, 'offense': row.offense, 'incidents': row.incidents
                     } for row in results]

        dataToReturn = {'data': data_list, 'state': state_list, 'year': year_list}
        
        # return jsonify(dataToReturn)
        return dataToReturn   
    
    except Exception as e:
        print("Error accessing the table:", str(e))
        # return jsonify({"error": "Table access failed"}), 500

In [68]:
# Preview data from list for notebook only
data = get_data_offenses()

data['data'][:4]

[{'year': 2009, 'state': 'Alabama', 'offense': 'Arson', 'incidents': 1},
 {'year': 2009,
  'state': 'Alabama',
  'offense': 'Destruction/Damage/Vandalism of Property',
  'incidents': 4},
 {'year': 2009, 'state': 'Alabama', 'offense': 'Intimidation', 'incidents': 4},
 {'year': 2009,
  'state': 'Alaska',
  'offense': 'Aggravated Assault',
  'incidents': 2}]